## Data Exploration (ML50-2023) - Mateus

### __Importing libraries__

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Data path : ../Data/test.csv
test_path = os.path.join('..', 'Data', 'test.csv')
train_path = os.path.join('..', 'Data', 'train.csv')
icd_9_path = os.path.join('..', 'Data', 'icd9_codes.txt')

# Read data
test = pd.read_csv(test_path, index_col='encounter_id')
train = pd.read_csv(train_path, index_col='encounter_id')
icd_9 = pd.read_csv(icd_9_path, sep=',', encoding='ISO-8859-1', index_col=0)
icd_9['3 digit code'] = icd_9.index.str[:3]
icd_9.set_index('3 digit code', inplace=True)
# Delete duplicates in the index
icd_9 = icd_9[~icd_9.index.duplicated(keep='first')]

test_ids = test.index
train_ids = train.index

In [3]:
# Data will have to be treated as a whole, so we will concatenate the test and training data
data = pd.concat([train, test], axis=0)

# Adding a 0 to the codes that have only two digits
data['primary_diagnosis'] = data['primary_diagnosis'].apply(lambda x: '0' + x if len(x) == 2 else x)
data['primary_diagnosis'] = data['primary_diagnosis'].apply(lambda x: '00' + x if len(x) == 1 and x!='?' else x)
data['primary_diagnosis'] = data['primary_diagnosis'].apply(lambda x: x if len(x) == 3 else x[:3])
data['secondary_diagnosis'] = data['secondary_diagnosis'].apply(lambda x: '0' + x if len(x) == 2 else x)
data['secondary_diagnosis'] = data['secondary_diagnosis'].apply(lambda x: '00' + x if len(x) == 1 and x != '?' else x)
data['secondary_diagnosis'] = data['secondary_diagnosis'].apply(lambda x: x if len(x) == 3 else x[:3])
data['additional_diagnosis'] = data['additional_diagnosis'].apply(lambda x: '0' + x if len(x) == 2 else x)
data['additional_diagnosis'] = data['additional_diagnosis'].apply(lambda x: '00' + x if len(x) == 1 and x != '?' else x)
data['additional_diagnosis'] = data['additional_diagnosis'].apply(lambda x: x if len(x) == 3 else x[:3])


# This will ease our work later on
data['primary_diagnosis_description'] = data['primary_diagnosis'].map(icd_9['long_description'])
data['secondary_diagnosis_description'] = data['secondary_diagnosis'].map(icd_9['long_description'])
data['additional_diagnosis_description'] = data['additional_diagnosis'].map(icd_9['long_description'])

# Transform the target variable into a binary variable
data['readmitted_binary'] = data['readmitted_binary'].map({'No': 0, 'Yes': 1})
data['prescribed_diabetes_meds'] = data['prescribed_diabetes_meds'].map({'No': 0, 'Yes': 1})

In [4]:
data.head(8).T

encounter_id,533253,426224,634063,890610,654194,269878,182051,964239
country,USA,USA,USA,USA,USA,USA,USA,USA
patient_id,70110,29775006,80729253,2919042,84871971,279288,1566405,60052095
race,Caucasian,AfricanAmerican,Caucasian,AfricanAmerican,Caucasian,Caucasian,Caucasian,Other
gender,Female,Male,Female,Male,Female,Female,Female,Male
age,[70-80),[50-60),[60-70),[60-70),[70-80),[50-60),[50-60),[70-80)
weight,?,?,?,?,?,?,?,?
payer_code,?,?,?,MC,HM,?,UN,MC
outpatient_visits_in_previous_year,0,0,0,0,1,0,0,0
emergency_visits_in_previous_year,0,0,0,0,0,0,0,0
inpatient_visits_in_previous_year,2,0,1,1,0,0,0,0


### __I. Taking care of missing data__
Lets check for which values are missing (in percentage of the total dataframe size)

In [5]:
def get_missing_per(data):
    isna = (100 * data.isna().sum().sort_values(ascending=False) / len(data))
    isna = isna[isna != 0]

    missing_counts = 100 *data.apply(lambda x: x.value_counts().get('?', 0)).sort_values(ascending=False) / len(data)
    missing_counts = missing_counts[missing_counts != 0]

    return pd.concat([isna, missing_counts], axis=0).sort_values(ascending=False)

get_missing_per(data)

weight                              96.858479
glucose_test_result                 94.746772
a1c_test_result                     83.277322
medical_specialty                   49.082208
payer_code                          39.557416
readmitted_multiclass               30.000197
readmitted_binary                   30.000197
admission_source                     6.663326
admission_type                       5.199182
race                                 4.999705
age                                  4.999705
discharge_disposition                3.626948
race                                 2.135291
additional_diagnosis                 1.398306
additional_diagnosis_description     1.398306
secondary_diagnosis_description      0.351787
secondary_diagnosis                  0.351787
primary_diagnosis_description        0.020636
primary_diagnosis                    0.020636
dtype: float64

#### __0. Taking care of variables with high % of missing values__
We have many variables with a missing value count upwards of 30%, which should be removed. Lets check how they are, before removing them.

##### 1. *Weight*

In [6]:
data.groupby('weight')['readmitted_binary'].mean().sort_values(ascending=False)

weight
[0-25)       0.171429
[175-200)    0.142857
?            0.111697
[50-75)      0.111635
[100-125)    0.111359
[75-100)     0.109325
[25-50)      0.104478
[150-175)    0.095238
[125-150)    0.052083
>200         0.000000
Name: readmitted_binary, dtype: float64

In [7]:
data.groupby('weight')['prescribed_diabetes_meds'].mean().sort_values(ascending=False)

weight
>200         1.000000
[175-200)    0.909091
[0-25)       0.791667
?            0.772677
[125-150)    0.765517
[150-175)    0.714286
[75-100)     0.694611
[100-125)    0.684800
[50-75)      0.665552
[25-50)      0.628866
Name: prescribed_diabetes_meds, dtype: float64

Although there seems to be some variance in the weight, with respect to the target variable, the missing values are too many to be imputed. We will remove this variable, as it might introduce bias in the model.

##### 2. *Glucose test results*

In [8]:
print(data.groupby('glucose_test_result')['readmitted_binary'].mean().sort_values(ascending=False))
print(data.groupby('a1c_test_result')['readmitted_binary'].mean().sort_values(ascending=False))

glucose_test_result
>300    0.142684
>200    0.130806
Norm    0.121816
Name: readmitted_binary, dtype: float64
a1c_test_result
>7      0.101551
Norm    0.099914
>8      0.096231
Name: readmitted_binary, dtype: float64


These two variables are very important to diagnose diabetes, and we can assume that if the test is Nan, it was not performed (probably because the patient was already known to be diabetic). We will impute the missing values with Not Performed.

We are also going to rename the columns to a1c_test_result and glucose_test_result, for better readability, and according to the following article:
<br> '*[A] diagnosis of diabetes can be made when the A1C exceeds 6.5% or when a random glycose level in a patient with classic symptoms exceeds 200 mg/dL.*' (https://www.ncbi.nlm.nih.gov/books/NBK551501/), meaning:
1. We can encode the a1c_test_result as 0 for 'Norma' and 1 for 'Abnormal', since the a1c values range from Norm, >7, >8. 
2. We will also use the glycose reading, with a threshold of >300mg/dL to encode it as 1, and 0 otherwise.
3. We can then join both dataframes, creating a new column name diabetes

In [9]:
glucose_map = {'Norm': 'Normal', '>200': 'Probably diabetic', '>300': 'Diabetic'}
a1c_map = {'Norm': 'Normal', '>7': 'Diabetic', '>8': 'Diabetic'}
data['glucose_test_result'] = data['glucose_test_result'].map(glucose_map).fillna('Not tested')
data['a1c_test_result'] = data['a1c_test_result'].map(a1c_map).fillna('Not tested')
print(data['glucose_test_result'].value_counts(), '\n')
print(data['a1c_test_result'].value_counts())

glucose_test_result
Not tested           96420
Normal                2597
Probably diabetic     1485
Diabetic              1264
Name: count, dtype: int64 

a1c_test_result
Not tested    84748
Diabetic      12028
Normal         4990
Name: count, dtype: int64


##### 3. *Medical specialty*

In [10]:
data.groupby('medical_specialty')['readmitted_binary'].mean().sort_values(ascending=False).head(5)

medical_specialty
Resident                          0.500000
AllergyandImmunology              0.500000
Pediatrics-Hematology-Oncology    0.250000
Hematology                        0.207547
Hematology/Oncology               0.202614
Name: readmitted_binary, dtype: float64

In [11]:
data[['medical_specialty','primary_diagnosis_description']].value_counts(dropna=False).head(10)

medical_specialty  primary_diagnosis_description                                                                               
?                  Diabetes mellitus without mention of complication - type II or unspecified type - not stated as uncontrolled    4179
                   Congestive heart failure - unspecified                                                                          3552
                   Coronary atherosclerosis of unspecified type of vessel - native or graft                                        2821
                   Acute myocardial infarction of anterolateral wall - episode of care unspecified                                 1998
                   Respiratory abnormality - unspecified                                                                           1876
Cardiology         Coronary atherosclerosis of unspecified type of vessel - native or graft                                        1839
?                  Pneumonia - organism unspecified     

Medical speciality is a tricky one. We have a lot of missing values, but it can explain some of the variance in the target variable. We could try to input the missing values with the help of the primary_diagnosis variable, but then we would just have the same data twice. We will remove this variable.

##### 4. *Insurance provider code*

In [12]:
data['payer_code'].value_counts(dropna=False).head(5)

payer_code
?     40256
MC    32439
HM     6274
SP     5007
BC     4655
Name: count, dtype: int64

In [13]:
data.groupby('payer_code')['readmitted_binary'].mean().sort_values(ascending=False).head(10)

payer_code
SI    0.133333
MP    0.129630
MD    0.125201
MC    0.118900
OG    0.115226
?     0.115209
DM    0.108466
SP    0.100537
HM    0.097939
BC    0.093864
Name: readmitted_binary, dtype: float64

There doesn't seem to be enough explained variance in the target variable to justify the use of this variable. We will remove it.

##### 5. Removing variables mentioned above

In [14]:
data = data.drop(['weight', 'payer_code', 'medical_specialty'], axis=1, errors='ignore')

#### __1. Taking care of the target variable__

Lets check if we can extract information from readmitted_binary to fill in the missing values in readmitted_multiclass (and *vice-versa*).

In [15]:
get_missing_per(data)

readmitted_binary                   30.000197
readmitted_multiclass               30.000197
admission_source                     6.663326
admission_type                       5.199182
race                                 4.999705
age                                  4.999705
discharge_disposition                3.626948
race                                 2.135291
additional_diagnosis_description     1.398306
additional_diagnosis                 1.398306
secondary_diagnosis_description      0.351787
secondary_diagnosis                  0.351787
primary_diagnosis_description        0.020636
primary_diagnosis                    0.020636
dtype: float64

In [16]:
# Our target variable is readmitted_binary and readmitted_multiclass. From multiclass we can derive binary, so lets check if they're overlapping
data['readmitted_binary'][data['readmitted_binary'].isna()].index.equals(data['readmitted_multiclass'][data['readmitted_binary'].isna()].index)

True

Since the readmitted_binary missing values coincide with the readmitted_multiclass missing values, we can't use one to fill in the other. We will remove the missing values.

In [17]:
# They are indeed, so we have to drop the missing values from both
data = data.dropna(subset=['readmitted_binary', 'readmitted_multiclass'])

#### __2. Handling the admission variables__

- What are the admission_sources values? What about the admission types?

In [18]:
# Lets now check our isna() values again
get_missing_per(data)

admission_source                    6.623056
admission_type                      5.202426
age                                 4.993262
race                                4.989050
discharge_disposition               3.635802
race                                2.128137
additional_diagnosis_description    1.415015
additional_diagnosis                1.415015
secondary_diagnosis_description     0.367792
secondary_diagnosis                 0.367792
primary_diagnosis_description       0.022461
primary_diagnosis                   0.022461
dtype: float64

In [64]:
print(data.groupby('admission_type', dropna=False)['readmitted_binary'].agg(['mean', 'count']).sort_values(by='mean', ascending=False))
print(data.groupby('admission_source', dropna=False)['readmitted_binary'].agg(['mean', 'count']).sort_values(by='mean', ascending=False))

                    mean  count
admission_type                 
1               0.114938  37742
5               0.112101  13024
6               0.108312   7026
0               0.103853  13211
3               0.088785    214
2               0.000000      6
4               0.000000     13
                      mean  count
admission_source                 
0                 0.181818     11
12                0.155039    129
3                 0.140187    107
7                 0.127731    595
10                0.125000      8
1                 0.116620  40319
16                0.106825   4718
11                0.105263    779
4                 0.105184  20678
8                 0.097311   1562
14                0.095964   2230
15                0.079545     88
6                 0.000000      2
5                 0.000000      1
9                 0.000000      7
2                 0.000000      1
13                0.000000      1


Not Available is the same as Nan, so lets replace Not available with nan for now

In [56]:
data['admission_type'] = data['admission_type'].apply(lambda x: None if x == 'Not Available' else x)
data['admission_source'] = data['admission_source'].apply(lambda x: None if x == ' Not Available' else x)

Lets use a predictive model to impute the Not Available values in admission_type. Before we do that, we need to encode the categorical variables.

In [72]:
from sklearn.preprocessing import LabelEncoder
# Lets encode the admission_type
encoder_type = LabelEncoder()
encoder_source = LabelEncoder()
data['admission_type'] = encoder_type.fit_transform(data['admission_type'])
data['admission_source'] = encoder_source.fit_transform(data['admission_source'])
print(encoder_source.classes_)
print(encoder_type.classes_)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
[0 1 2 3 4 5 6]


No, we can't, so we have to find a way to fill in the missing values. Lets try to use a predictive model to fill in the missing values, like the KNNImputer.

In [73]:
encoder_source.classes_     CRFVTB7N8M90 BVC    1

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16])